In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 1
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.19 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_3S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1


C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torchvision\ops\misc.py:120: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(
Downloading: "https://github.com/fschmid56/EfficientAT/releases/download/v0.0.1/mn40_as_ext_mAP_487.pt" to resources\mn40_as_ext_mAP_487.pt
100.0%


Number of classes defined: 10, but try to load pre-trained layer with logits: 527
Dropping last layer.
Error(s) in loading state_dict for MobileNetV3:
	Missing key(s) in state_dict: "classifier.5.weight", "classifier.5.bias". 
Loading weights pre-trained weights in a non-strict manner.
MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:867.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


Train Epoch: 1/50 Train_Loss: 2.3299012184143066 Train_acc: 8.750000%
Train Epoch: 2/50 Train_Loss: 2.318361759185791 Train_acc: 10.520834%
Train Epoch: 3/50 Train_Loss: 2.276566982269287 Train_acc: 11.666667%
Train Epoch: 4/50 Train_Loss: 2.2907581329345703 Train_acc: 12.395834%
Train Epoch: 5/50 Train_Loss: 2.246807098388672 Train_acc: 15.208334%
Train Epoch: 6/50 Train_Loss: 2.1754751205444336 Train_acc: 23.541668%
Train Epoch: 7/50 Train_Loss: 2.0535383224487305 Train_acc: 37.083336%
Train Epoch: 8/50 Train_Loss: 1.8545161485671997 Train_acc: 59.062504%
Train Epoch: 9/50 Train_Loss: 1.6249538660049438 Train_acc: 79.687508%
Train Epoch: 10/50 Train_Loss: 1.2900387048721313 Train_acc: 90.520836%
Train Epoch: 11/50 Train_Loss: 0.985253095626831 Train_acc: 92.708336%
Train Epoch: 12/50 Train_Loss: 0.8409008383750916 Train_acc: 94.791672%
Train Epoch: 13/50 Train_Loss: 0.6108344793319702 Train_acc: 95.625008%
Train Epoch: 14/50 Train_Loss: 0.5165326595306396 Train_acc: 96.041672%
Train 

Train Epoch: 1/50 Train_Loss: 2.2807228565216064 Train_acc: 13.645834%
Train Epoch: 2/50 Train_Loss: 2.2947449684143066 Train_acc: 13.020834%
Train Epoch: 3/50 Train_Loss: 2.2818896770477295 Train_acc: 13.645834%
Train Epoch: 4/50 Train_Loss: 2.2395570278167725 Train_acc: 15.833334%
Train Epoch: 5/50 Train_Loss: 2.2186150550842285 Train_acc: 18.437502%
Train Epoch: 6/50 Train_Loss: 2.1784050464630127 Train_acc: 25.000002%
Train Epoch: 7/50 Train_Loss: 2.006603956222534 Train_acc: 40.000004%
Train Epoch: 8/50 Train_Loss: 1.8443115949630737 Train_acc: 64.166672%
Train Epoch: 9/50 Train_Loss: 1.633318305015564 Train_acc: 81.041672%
Train Epoch: 10/50 Train_Loss: 1.352554440498352 Train_acc: 88.645836%
Train Epoch: 11/50 Train_Loss: 1.0263234376907349 Train_acc: 93.541672%
Train Epoch: 12/50 Train_Loss: 0.7895123362541199 Train_acc: 95.312508%
Train Epoch: 13/50 Train_Loss: 0.8051596879959106 Train_acc: 96.145836%
Train Epoch: 14/50 Train_Loss: 0.5319217443466187 Train_acc: 96.562508%
Trai

Train Epoch: 1/50 Train_Loss: 2.309483528137207 Train_acc: 8.541667%
Train Epoch: 2/50 Train_Loss: 2.2760119438171387 Train_acc: 9.375001%
Train Epoch: 3/50 Train_Loss: 2.331205368041992 Train_acc: 9.583334%
Train Epoch: 4/50 Train_Loss: 2.269784927368164 Train_acc: 12.187501%
Train Epoch: 5/50 Train_Loss: 2.281858444213867 Train_acc: 16.041668%
Train Epoch: 6/50 Train_Loss: 2.2299463748931885 Train_acc: 21.666668%
Train Epoch: 7/50 Train_Loss: 2.014017343521118 Train_acc: 36.875004%
Train Epoch: 8/50 Train_Loss: 1.838814377784729 Train_acc: 59.895836%
Train Epoch: 9/50 Train_Loss: 1.6458207368850708 Train_acc: 78.229172%
Train Epoch: 10/50 Train_Loss: 1.348774790763855 Train_acc: 89.062508%
Train Epoch: 11/50 Train_Loss: 1.039206624031067 Train_acc: 92.500008%
Train Epoch: 12/50 Train_Loss: 0.8555269837379456 Train_acc: 94.270836%
Train Epoch: 13/50 Train_Loss: 0.6217795014381409 Train_acc: 95.416672%
Train Epoch: 14/50 Train_Loss: 0.375049352645874 Train_acc: 95.625008%
Train Epoch: 

Train Epoch: 1/50 Train_Loss: 2.42366623878479 Train_acc: 3.437500%
Train Epoch: 2/50 Train_Loss: 2.3902432918548584 Train_acc: 5.000000%
Train Epoch: 3/50 Train_Loss: 2.367520332336426 Train_acc: 5.208333%
Train Epoch: 4/50 Train_Loss: 2.3268051147460938 Train_acc: 6.875000%
Train Epoch: 5/50 Train_Loss: 2.3328166007995605 Train_acc: 9.583334%
Train Epoch: 6/50 Train_Loss: 2.211663246154785 Train_acc: 14.479167%
Train Epoch: 7/50 Train_Loss: 2.1183922290802 Train_acc: 24.583334%
Train Epoch: 8/50 Train_Loss: 1.9116588830947876 Train_acc: 42.500004%
Train Epoch: 9/50 Train_Loss: 1.6772128343582153 Train_acc: 68.645836%
Train Epoch: 10/50 Train_Loss: 1.410670518875122 Train_acc: 85.937508%
Train Epoch: 11/50 Train_Loss: 1.1226025819778442 Train_acc: 91.354172%
Train Epoch: 12/50 Train_Loss: 0.8058998584747314 Train_acc: 93.020836%
Train Epoch: 13/50 Train_Loss: 0.589821994304657 Train_acc: 93.958336%
Train Epoch: 14/50 Train_Loss: 0.41494038701057434 Train_acc: 94.895836%
Train Epoch: 1

Train Epoch: 1/50 Train_Loss: 2.3901736736297607 Train_acc: 7.395833%
Train Epoch: 2/50 Train_Loss: 2.327901840209961 Train_acc: 8.854167%
Train Epoch: 3/50 Train_Loss: 2.3247976303100586 Train_acc: 7.604167%
Train Epoch: 4/50 Train_Loss: 2.292271614074707 Train_acc: 10.104167%
Train Epoch: 5/50 Train_Loss: 2.2849690914154053 Train_acc: 13.437501%
Train Epoch: 6/50 Train_Loss: 2.207003355026245 Train_acc: 19.687502%
Train Epoch: 7/50 Train_Loss: 2.0850558280944824 Train_acc: 31.666668%
Train Epoch: 8/50 Train_Loss: 1.871203899383545 Train_acc: 52.916668%
Train Epoch: 9/50 Train_Loss: 1.6517523527145386 Train_acc: 75.416672%
Train Epoch: 10/50 Train_Loss: 1.3250120878219604 Train_acc: 87.604172%
Train Epoch: 11/50 Train_Loss: 1.0920286178588867 Train_acc: 92.500008%
Train Epoch: 12/50 Train_Loss: 0.7581101059913635 Train_acc: 94.583336%
Train Epoch: 13/50 Train_Loss: 0.6277114748954773 Train_acc: 95.520836%
Train Epoch: 14/50 Train_Loss: 0.4866810441017151 Train_acc: 96.250008%
Train Ep

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()